In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myproj').getOrCreate()
data = spark.read.csv('/FileStore/tables/UCI_Credit_Card.csv',inferSchema=True,header=True)
data.printSchema()

root
-- ID: integer (nullable = true)
-- LIMIT_BAL: double (nullable = true)
-- SEX: integer (nullable = true)
-- EDUCATION: integer (nullable = true)
-- MARRIAGE: integer (nullable = true)
-- AGE: integer (nullable = true)
-- PAY_0: integer (nullable = true)
-- PAY_2: integer (nullable = true)
-- PAY_3: integer (nullable = true)
-- PAY_4: integer (nullable = true)
-- PAY_5: integer (nullable = true)
-- PAY_6: integer (nullable = true)
-- BILL_AMT1: double (nullable = true)
-- BILL_AMT2: double (nullable = true)
-- BILL_AMT3: double (nullable = true)
-- BILL_AMT4: double (nullable = true)
-- BILL_AMT5: double (nullable = true)
-- BILL_AMT6: double (nullable = true)
-- PAY_AMT1: double (nullable = true)
-- PAY_AMT2: double (nullable = true)
-- PAY_AMT3: double (nullable = true)
-- PAY_AMT4: double (nullable = true)
-- PAY_AMT5: double (nullable = true)
-- PAY_AMT6: double (nullable = true)
-- default.payment.next.month: integer (nullable = true)

Data Dictionary
1.	ID: ID of each client
2.	LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit)
3.	SEX: Gender (1=male, 2=female)
4.	EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
5.	MARRIAGE: Marital status (1=married, 2=single, 3=others)
6.	Age: Age in years
7.	PAY_0 ------ PAY_6 (6 features): Repayment status from April, 2005 to September, 2005(-1=pay duly, 0=not delay, other number = the number of months for payment delay)
8.	BILL_AMT1 ------ BILL_AMT6 (6 features): Amount of bill statement from April, 2005 to September 2005(NT dollar)
9.	PAY_AMT1: Amount of previous payment from April, 2005 to September 2005(NT dollar)
10.	Default. Payment. next. Month: default payment (1=yes, 2=no)

In [0]:
data = data.withColumnRenamed("PAY_0", "PAY_1")
data = data.withColumnRenamed("default.payment.next.month", "Default")

Change data types

In [0]:
from pyspark.sql.types import StructField,StringType,IntegerType,DoubleType,StructType
# if the second element is True means it could have null cells
df_schema = StructType([
    StructField('ID', StringType(), True),
    StructField('LIMIT_BAL', DoubleType(), True),
    StructField('SEX', StringType(), True),
    StructField('EDUCATION', StringType(), True),
    StructField('MARRIAGE', StringType(), True),
    StructField('AGE', IntegerType(), True),
    StructField('PAY_1', IntegerType(), True),
    StructField('PAY_2', IntegerType(), True),
    StructField('PAY_3', IntegerType(), True),
    StructField('PAY_4', IntegerType(), True),
    StructField('PAY_5', IntegerType(), True),
    StructField('PAY_6', IntegerType(), True),
    StructField('BILL_AMT1', DoubleType(), True),
    StructField('BILL_AMT2', DoubleType(), True),
    StructField('BILL_AMT3', DoubleType(), True),
    StructField('BILL_AMT4', DoubleType(), True),
    StructField('BILL_AMT5', DoubleType(), True),
    StructField('BILL_AMT6', DoubleType(), True),
    StructField('PAY_AMT1', DoubleType(), True),
    StructField('PAY_AMT2', DoubleType(), True),
    StructField('PAY_AMT3', DoubleType(), True),
    StructField('PAY_AMT4', DoubleType(), True),
    StructField('PAY_AMT5', DoubleType(), True),
    StructField('PAY_AMT6', DoubleType(), True),
    StructField('Default', IntegerType(), True),
    ])
data = spark.createDataFrame(data.collect(),schema = df_schema)
data.printSchema()

root
-- ID: string (nullable = true)
-- LIMIT_BAL: double (nullable = true)
-- SEX: string (nullable = true)
-- EDUCATION: string (nullable = true)
-- MARRIAGE: string (nullable = true)
-- AGE: integer (nullable = true)
-- PAY_1: integer (nullable = true)
-- PAY_2: integer (nullable = true)
-- PAY_3: integer (nullable = true)
-- PAY_4: integer (nullable = true)
-- PAY_5: integer (nullable = true)
-- PAY_6: integer (nullable = true)
-- BILL_AMT1: double (nullable = true)
-- BILL_AMT2: double (nullable = true)
-- BILL_AMT3: double (nullable = true)
-- BILL_AMT4: double (nullable = true)
-- BILL_AMT5: double (nullable = true)
-- BILL_AMT6: double (nullable = true)
-- PAY_AMT1: double (nullable = true)
-- PAY_AMT2: double (nullable = true)
-- PAY_AMT3: double (nullable = true)
-- PAY_AMT4: double (nullable = true)
-- PAY_AMT5: double (nullable = true)
-- PAY_AMT6: double (nullable = true)
-- Default: integer (nullable = true)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import when, count, col
df = data
na_report=df.select([count(when(isnull(c), c)).alias(c) for c in df.columns])# dimensions of the dataframe
print("Number of Rows: ",df.count() ,"   Number of Columns: ", len(df.columns))
# we should handle the missing values by imputation unless too many of them are emtpy
# na.drop() method is not recommended for avoiding biased except for the dependent variable
na_report.show()

Number of Rows: 30000 Number of Columns: 25
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
 ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|Default|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+

In [0]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+----------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+
summary| ID| LIMIT_BAL| SEX| EDUCATION| MARRIAGE| AGE| PAY_1| PAY_2| PAY_3| PAY_4| PAY_5| PAY_6| BILL_AMT1| BILL_AMT2| BILL_AMT3| BILL_AMT4| BILL_AMT5| BILL_AMT6| PAY_AMT1| PAY_AMT2| PAY_AMT3| PAY_AMT4| PAY_AMT5| PAY_AMT6| Default|
+-------+-----------------+------------------+------------------+------------------+------------------+----------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+
 count| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000| 30000|
 mean| 15000.5|167484.32266666667|1.6037333333333332|1.8531333333333333|1.5518666666666667| 35.4855| -0.0167|-0.13376666666666667| -0.1662|-0.22066666666666668| -0.2662| -0.2911| 51223.3309|49179.07516666667| 47013.1548| 43262.94896666666|40311.40096666667| 38871.7604| 5663.5805| 5921.1635| 5225.6815|4826.076866666666| 4799.387633333334| 5215.502566666667| 0.2212|
 stddev|8660.398374208891|129747.66156720239|0.4891291960902606|0.7903486597207294|0.5219696006132485|9.21790406809019|1.1238015279973346| 1.1971859730345529|1.1968675684465735| 1.1691386224023377|1.1331874060027485|1.1499876256079022|73635.86057552954|71173.76878252836|69349.38742703685|64332.856133916335|60797.15577026485|59554.10753674574|16563.280354025774|23040.87040205723|17606.961469803104| 15666.159744032|15278.305679144783|17777.465775435303|0.4150618056909332|
 min| 1| 10000.0| 1| 0| 0| 21| -2| -2| -2| -2| -2| -2| -165580.0| -69777.0| -157264.0| -170000.0| -81334.0| -339603.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0|
 max| 9999| 1000000.0| 2| 6| 3| 79| 8| 8| 8| 8| 8| 8| 964511.0| 983931.0| 1664089.0| 891586.0| 927171.0| 961664.0| 873552.0| 1684259.0| 896040.0| 621000.0| 426529.0| 528666.0| 1|
+-------+-----------------+------------------+------------------+------------------+------------------+----------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+

Drop abnormal data in "EDUCATION" and "MARRIAGE" comlumns

In [0]:
df.select('MARRIAGE','EDUCATION').describe().show()

+-------+------------------+------------------+
summary| MARRIAGE| EDUCATION|
+-------+------------------+------------------+
 count| 30000| 30000|
 mean|1.5518666666666667|1.8531333333333333|
 stddev|0.5219696006132485|0.7903486597207294|
 min| 0| 0|
 max| 3| 6|
+-------+------------------+------------------+

In [0]:
df = df.filter((df.MARRIAGE != '3')&(df.MARRIAGE != '0') &(df.EDUCATION != '0') &  (df.EDUCATION != '5') & (df.EDUCATION != '6'))

In [0]:
df.select('MARRIAGE','EDUCATION').describe().show()

+-------+------------------+------------------+
summary| MARRIAGE| EDUCATION|
+-------+------------------+------------------+
 count| 29283| 29283|
 mean|1.5397671003653997| 1.8114605743947|
 stddev|0.4984245794401454|0.7094951994131745|
 min| 1| 1|
 max| 2| 4|
+-------+------------------+------------------+

Visualizations

In [0]:
df.select("EDUCATION","Default","ID").groupBy("EDUCATION","Default").agg(count("ID")).orderBy('EDUCATION').display()

EDUCATION,Default,count(ID)
1,0,8508
1,1,2023
2,0,10577
2,1,3285
3,1,1206
3,0,3564
4,0,113
4,1,7
